In [1]:
import pandas as pd
from scipy import stats
from matplotlib import cm, colors
from mpl_toolkits.axes_grid1 import ImageGrid
import json
import pickle
import csv
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import glob, os
import random
import pickle
import re

import tensorflow as tf
from tensorflow import keras

2023-05-30 20:58:23.908200: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-30 20:58:33.411032: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-05-30 20:58:55.468718: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/jet/home/jlin96/.conda/envs/tf2/lib/
2023-05-30 20:58:55.469214: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plug

In [2]:
# in/out variable lists
vars_mli = ['state_t','state_q0001','state_ps','pbuf_SOLIN', 'pbuf_LHFLX', 'pbuf_SHFLX']
vars_mlo = ['ptend_t','ptend_q0001','cam_out_NETSW','cam_out_FLWDS','cam_out_PRECSC','cam_out_PRECC','cam_out_SOLS','cam_out_SOLL','cam_out_SOLSD','cam_out_SOLLD']

In [3]:
!pwd

/ocean/projects/atm200007p/jlin96/neurips_proj/e3sm_train_npy


In [4]:
# change these for your HPC and repo path
data_path = '/ocean/projects/atm200007p/walrus/for_jerry/train/'
norm_path = '/ocean/projects/atm200007p/jlin96/neurips_proj/mooers_metrics/norm_factors/'
grid_path = '/ocean/projects/atm200007p/jlin96/neurips_proj/mooers_metrics/test_data/E3SM-MMF_ne4_grid-info.orig.nc'
save_path = '/ocean/projects/atm200007p/jlin96/neurips_proj/e3sm_train_npy/'

In [5]:
# new dataset generator function
# that has new options (latlim, lonlim)

mli_mean = xr.open_dataset(norm_path + 'mli_mean.nc')
mli_min = xr.open_dataset(norm_path + 'mli_min.nc')
mli_max = xr.open_dataset(norm_path + 'mli_max.nc')
mlo_scale = xr.open_dataset(norm_path + 'mlo_scale.nc')
ne4_grid_info = xr.open_dataset(grid_path)

In [6]:
def ls(data_path = ""):
    return os.popen(" ".join(["ls", data_path])).read().splitlines()

def concatenate_arrays(xrdata, vars):
    return np.concatenate([np.atleast_1d(xrdata[var].values) for var in vars])

In [7]:
mli_mean_npy = concatenate_arrays(mli_mean, vars_mli)
mli_min_npy = concatenate_arrays(mli_min, vars_mli)
mli_max_npy = concatenate_arrays(mli_max, vars_mli)
mlo_scale_npy = concatenate_arrays(mlo_scale, vars_mlo)

# Debugging

In [97]:
latlim=[-999,999]
lonlim=[-999,999]

def showme_i(file):
    '''
    This function reads in a file and returns an xarray dataset with the input variables for the emulator.
    '''
    ds = xr.open_dataset(file, engine = 'netcdf4')
    ds = ds[vars_mli]
    ds = ds.merge(ne4_grid_info[['lat','lon']])
    ds = ds.where((ds['lat']>latlim[0])*(ds['lat']<latlim[1]),drop=True)
    ds = ds.where((ds['lon']>lonlim[0])*(ds['lon']<lonlim[1]),drop=True)
    return(ds)

def showme_o(file):
    '''
    This function reads in a file and returns an xarray dataset with the output variables for the emulator.
    '''
    # read mli
    ds = xr.open_dataset(file, engine='netcdf4')
    ds = ds[vars_mli]
    ds = ds.merge(ne4_grid_info[['lat','lon']])
    ds = ds.where((ds['lat']>latlim[0])*(ds['lat']<latlim[1]),drop=True)
    ds = ds.where((ds['lon']>lonlim[0])*(ds['lon']<lonlim[1]),drop=True)

    # read mlo
    dso = xr.open_dataset(file.replace('.mli.','.mlo.'), engine='netcdf4')
    dso = dso.merge(ne4_grid_info[['lat','lon']])
    dso = dso.where((dso['lat']>latlim[0])*(dso['lat']<latlim[1]),drop=True)
    dso = dso.where((dso['lon']>lonlim[0])*(dso['lon']<lonlim[1]),drop=True)

    # make mlo variales: ptend_t and ptend_q0001
    dso['ptend_t'] = (dso['state_t'] - ds['state_t'])/1200 # T tendency [K/s]
    dso['ptend_q0001'] = (dso['state_q0001'] - ds['state_q0001'])/1200 # Q tendency [kg/kg/s]
    dso = dso[vars_mlo]

    # normalization, scaling
    # dso = dso*mlo_scale

    # stack
    #dso = dso.stack({'batch':{'sample','ncol'}})
    # dso = dso.stack({'batch':{'ncol'}})
    # dso = dso.to_stacked_array("mlvar", sample_dims=["batch"], name='mlo')
    return dso

def load_nc_dir_with_generator_test(filelist:list, latlim=[-999,999], lonlim=[-999,999]):
    '''
    This function works as a dataloader when training the emulator with raw netCDF files.
    '''
    def gen():
        for file in filelist:
            
            # read mli
            ds = xr.open_dataset(file, engine='netcdf4')
            ds = ds[vars_mli]
            ds = ds.merge(ne4_grid_info[['lat','lon']])
            ds = ds.where((ds['lat']>latlim[0])*(ds['lat']<latlim[1]),drop=True)
            ds = ds.where((ds['lon']>lonlim[0])*(ds['lon']<lonlim[1]),drop=True)
            
            # read mlo
            dso = xr.open_dataset(file.replace('.mli.','.mlo.'), engine='netcdf4')
            dso = dso.merge(ne4_grid_info[['lat','lon']])
            dso = dso.where((dso['lat']>latlim[0])*(dso['lat']<latlim[1]),drop=True)
            dso = dso.where((dso['lon']>lonlim[0])*(dso['lon']<lonlim[1]),drop=True)
            
            # make mlo variales: ptend_t and ptend_q0001
            dso['ptend_t'] = (dso['state_t'] - ds['state_t'])/1200 # T tendency [K/s]
            dso['ptend_q0001'] = (dso['state_q0001'] - ds['state_q0001'])/1200 # Q tendency [kg/kg/s]
            dso = dso[vars_mlo]
            
            # normalization, scaling
            ds = (ds-mli_mean)/(mli_max-mli_min)
            dso = dso*mlo_scale

            # stack
            #ds = ds.stack({'batch':{'sample','ncol'}})
            ds = ds.stack({'batch':{'ncol'}})
            ds = ds.to_stacked_array("mlvar", sample_dims=["batch"], name='mli')
            #dso = dso.stack({'batch':{'sample','ncol'}})
            dso = dso.stack({'batch':{'ncol'}})
            dso = dso.to_stacked_array("mlvar", sample_dims=["batch"], name='mlo')
            
            yield (ds.values, dso.values)

    return tf.data.Dataset.from_generator(
        gen,
        output_types=(tf.float64, tf.float64),
        output_shapes=((None,124),(None,128))
    )

def reshape_npy(var_arr, var_arr_dim):
    '''
    This function reshapes the saved numpy datasets such that time gets its own axis.
    Shape of output would be (timestep, lat lon combo, var_arr_dim)
    '''
    var_arr = var_arr.reshape((int(var_arr.shape[0]/384), 384, var_arr_dim))
    return(var_arr)

def unscale_ps(ps):
    '''
    This function unscales the pressure variable so that it can be used to create a pressure grid.
    '''
    return ps * (mli_max["state_ps"].values - mli_min["state_ps"].values) + mli_mean["state_ps"].values

def getdates(*args, stride_sample):
    '''
    This function takes in a list of file endings and returns a list of dates corresponding to the files.
    '''
    f_mli_data = []
    for arg in args:
        f_mli_data = f_mli_data + glob.glob(data_path + arg)
    f_mli_data = sorted(f_mli_data)[::stride_sample]
    dates = [re.sub("^.*mli\.", "", x) for x in f_mli_data]
    dates = [re.sub("\.nc$", "", x) for x in dates]
    repeat_dates = []
    for date in dates:
        for i in range(384):
            repeat_dates.append(date)
    return repeat_dates

# with open(save_path + 'indextolatlons.pkl', 'wb') as f:
#     pickle.dump(latlons, f)

In [12]:
# with open(save_path + 'train_input.npy', 'rb') as f:
#     train_input = np.load(f)

# with open(save_path + 'train_target.npy', 'rb') as f:
#     train_target = np.load(f)
    
with open(save_path + 'val_input_stride6.npy', 'rb') as f:
    val_input = np.load(f)

In [66]:
val_ps = val_input[:,120]
unscaled_val_ps = unscale_ps(val_ps)[:, np.newaxis]
num_samples = unscaled_val_ps.shape[0]
hyam_component = np.repeat(ne4_grid_info["hyam"].values[np.newaxis, :], num_samples, axis=0)*1e5
hybm_component = np.repeat(ne4_grid_info["hybm"].values[np.newaxis, :], num_samples, axis=0)*unscaled_val_ps
pressure_grid_val = hyam_component + hybm_component

with open(save_path + "pressures_val_stride6.npy", 'wb') as f:
    np.save(f, np.float32(pressure_grid_val))

In [98]:
val_dates = getdates('*/E3SM-MMF.mli.0008-0[23456789]-*-*.nc', '*/E3SM-MMF.mli.0008-1[012]-*-*.nc', '*/E3SM-MMF.mli.0009-01-*-*.nc', stride_sample = 6)

In [99]:
len(val_dates)

1681920

In [89]:
val_input.shape

(1681920, 124)

In [121]:
latlontime = {i: [(ne4_grid_info["lat"].values[i%384], ne4_grid_info["lon"].values[i%384]), val_dates[i]] for i in range(val_input.shape[0])}

In [122]:
with open(save_path + 'indextolatlontime.pkl', 'wb') as f:
    pickle.dump(latlontime, f)

In [123]:
print('finished')

finished
